In [12]:
import os
import pandas as pd

In [15]:
deg_result_path = "/home/hb/python/lof/deg/norman"
# len(os.listdir(deg_result_path))

deg_result_path = "/home/hb/python/lof/deg/norman"
print('현재까지 계산 완료된 DEG 데이터 개수:', len([x for x in os.listdir(deg_result_path) if '.csv' in x]), '개')

현재까지 계산 완료된 DEG 데이터 개수: 45 개


In [6]:
import pandas as pd
import os
os.chdir("/home/hb/python/lof/deg/norman")
CONDITION = 'TGFBR2_IGDCC3'
path = '/home/hb/python/lof/benchmark/sigCOM_lincs'
# xpr_info = pd.read_csv("/home/hb/python/lof/clue/info/siginfo_beta_전체.txt", sep='\t')
# xpr_info

In [126]:
import requests
import json

API_URL = "https://maayanlab.cloud/sigcom-lincs/metadata-api/signatures/find"

payload = {
    "filter": {
        "where": {
            "meta": {
                
            }
        },
        "limit": 2
    }
}

res = requests.post(API_URL, json=payload)
results = res.json()

print(json.dumps(results, indent=2))

[
  {
    "$validator": "/dcic/signature-commons-schema/v5/core/signature.json",
    "id": "fe1d3fe5-c3f5-5924-a8ad-5a11f8ca8291",
    "library": "8f1ff550-ece8-591d-a213-2763f854c008",
    "meta": {
      "md5": "8d312e26ba1d0f7e9291c994c7faf35b",
      "doid": "DOID:1909",
      "sha256": "155d5984e2b127b3070c2e5116db35687a5c864ec4c71b447773e9f24a3ee50a",
      "tissue": "skin of body",
      "anatomy": "UBERON:0002097",
      "cmap_id": "KDC002_A375_96H:TRCN0000073978:-666",
      "disease": "melanoma",
      "version": 1,
      "filename": "L1000_LINCS_DCIC_KDC002_A375_96H_N07_EIF3H.tsv",
      "local_id": "KDC002_A375_96H_N07_EIF3H",
      "cell_line": "A375",
      "pert_name": "EIF3H",
      "pert_time": "96 h",
      "pert_type": "shRNA",
      "$validator": "https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/l1000_signatures.json",
      "data_level": 5,
      "creation_time": "2021-05-16",
      "persistent_id": "https://lincs-dcic.s3.amazonaws.com/LINCS

In [83]:
import pandas as pd
import os
import requests
import json
os.chdir("/home/hb/python/lof/deg/norman")
CONDITION = 'TGFBR2_IGDCC3'
path = '/home/hb/python/lof/benchmark/sigCOM_lincs'

def deg_analysis(CONDITION, signature):
     signature = "l1000_xpr" or "LINCS consensus gene (CGS) knockdown signatures"
    if signature == 'XPR':
        signature = 'l1000_xpr'
    elif signature == 'CGS':
        signature = 'LINCS consensus gene (CGS) knockdown signatures'
    else:
        print('invalid signature')
        
    deg = pd.read_csv(f"{CONDITION}.csv")
    try: 
        deg = deg.drop(['Unnamed: 0'], axis=1)
    except:
        pass
    column = deg.columns[1]
    deg_up = deg.sort_values(f"{column}", ascending=False).dropna(axis=0).set_index("gene_name")
    deg_down = deg.sort_values(f"{column}", ascending=True).dropna(axis=0).set_index("gene_name")
    up_genes = list(deg_up.iloc[:600].index.values)
    down_genes = list(deg_down.iloc[:600].index.values)

    METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
    DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

    input_gene_set = {
        "up_genes": up_genes,
        "down_genes": down_genes
    }

    all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

    payload = {
        "filter": {
            "where": {
                "meta.symbol": {
                    "inq": all_genes
                }
            },
            "fields": ["id", "meta.symbol"]
        }
    }
    res = requests.post(METADATA_API + "entities/find", json=payload)
    entities = res.json()

    for_enrichment = {
        "up_entities": [],
        "down_entities": []
    }

    for e in entities:
        symbol = e["meta"]["symbol"]
        if symbol in input_gene_set["up_genes"]:
            for_enrichment["up_entities"].append(e["id"])
        elif symbol in input_gene_set["down_genes"]:
            for_enrichment["down_entities"].append(e["id"])
    for_enrichment['up_entities'] = for_enrichment['up_entities'][:500]
    for_enrichment['down_entities'] = for_enrichment['down_entities'][:500]
    print(len(for_enrichment['up_entities']), len(for_enrichment['down_entities']))

    payload = {
        "meta": {
            "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
            **for_enrichment
        },
        "type": "signature"
    }
    res = requests.post(METADATA_API + "user_input", json=payload)
    persistent_id = res.json()["id"]
    print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

    NUM_CASE = 120
    query = {
        **for_enrichment,
        "limit": NUM_CASE,
        "database": signature
    }
    #
    res = requests.post(DATA_API + "enrich/ranktwosided", json=query)
    results = res.json()

    # Optional, multiply z-down and direction-down with -1
    for i in results["results"]:
        i["z-down"] = -i["z-down"]
        i["direction-down"] = -i["direction-down"]

    results = results['results'][:NUM_CASE]
    sigids = {i['uuid']: i for i in results}

    payload = {
        "filter": {
            "where": {
                "id": {
                    "inq": list(sigids.keys())
                }
            }
        }
    }

    res = requests.post(METADATA_API + "signatures/find", json=payload)
    signatures = res.json()

    ## Merge the scores and the metadata
    for sig in signatures:
        uid = sig["id"]
        scores = sigids[uid]
        # scores.pop("uuid")
        sig["scores"] = scores

    for i in signatures:
        dict1 = i['meta']
        dict2 = i['scores']

        dict1.update(dict2)

    df_mimickers = pd.DataFrame([x['meta'] for x in signatures])
    file_name = f'Norman_K562_{CONDITION}'

    df_mimickers = df_mimickers.sort_values(by=['z-sum'], ascending=False)
    df_mimickers.index = [f'Top-{x}' for x in range(1, len(df_mimickers)+1)]
    df_mimickers.dropna(subset=['pert_name'], inplace=True)
    # df_mimickers.dropna(subset=['pert_name'], ).resbet_index(drop=True, )
    # df_mimickers.dropna(subset=['pert_name'], inplace=True)
    # df_mimickers['z-sum']
    mimickers_save_path = f"{path}/{file_name}.csv"
    df_mimickers.to_csv(mimickers_save_path)

    df_mimickers_top10 = df_mimickers.iloc[:10]
    df_mimickers_top30 = df_mimickers.iloc[:30]
    df_mimickers_top50 = df_mimickers.iloc[:50]
    df_mimickers_top100 = df_mimickers.iloc[:100]

    benchmark = {}

    for i, rank in zip([df_mimickers_top10, df_mimickers_top30, df_mimickers_top50, df_mimickers_top100], [10, 30, 50, 100]):
        pert_list = i.pert_name.unique()

        if CONDITION not in benchmark.keys():
            benchmark[CONDITION] = {}
        if len(CONDITION.split('_')) > 1:

            CONDITION_LIST = CONDITION.split('_')
            print(CONDITION_LIST)
            if len(set(CONDITION_LIST).intersection(pert_list)) > 0:
                print(f'TOP-{rank} Accuracy: True')
                benchmark[CONDITION][f'Top-{rank}'] = 1
            else:   
                print(f'Top-{rank} Accuracy: False')
                benchmark[CONDITION][f'Top-{rank}'] = 0
        else:
            if CONDITION in pert_list:
                
                print(f'TOP-{rank} Accuracy: True')
                benchmark[CONDITION][f'Top-{rank}'] = 1
            else: 
                print(f'Top-{rank} Accuracy: False')
                benchmark[CONDITION][f'Top-{rank}'] = 0
    new_benchmark_df = pd.DataFrame(benchmark)
    display(df_mimickers.head())

    if signature == 'l1000_xpr':


        benchmark_df = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_xpr_benchmark_Norman.csv", index_col=0)
        if CONDITION not in benchmark_df.columns:
            benchmark_df = pd.concat([benchmark_df, new_benchmark_df], axis=1)
            benchmark_df.to_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_xpr_benchmark_Norman.csv", index=True)
        else:
            print('already exists')
    elif signature == "LINCS consensus gene (CGS) knockdown signatures":
        benchmark_df = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_CGS_benchmark_Norman.csv", index_col=0)
        if CONDITION not in benchmark_df.columns:
            benchmark_df = pd.concat([benchmark_df, new_benchmark_df], axis=1)
            benchmark_df.to_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_CGS_benchmark_Norman.csv", index=True)
        else:
            print('already exists')
    return benchmark_df

In [91]:
import pandas as pd
import os
import requests
import json
os.chdir("/home/hb/python/lof/deg/norman")
CONDITION = 'TGFBR2_IGDCC3'
path = '/home/hb/python/lof/benchmark/sigCOM_lincs'
signature = 'CGS'
# def deg_analysis(CONDITION, signature):
#  signature = "l1000_xpr" or "LINCS consensus gene (CGS) knockdown signatures"
if signature == 'XPR':
    signature = 'l1000_xpr'
elif signature == 'CGS':
    signature = 'LINCS consensus gene (CGS) knockdown signatures'
else:
    print('invalid signature')
    
deg = pd.read_csv(f"{CONDITION}.csv")
try: 
    deg = deg.drop(['Unnamed: 0'], axis=1)
except:
    pass
column = deg.columns[1]
deg_up = deg.sort_values(f"{column}", ascending=False).dropna(axis=0).set_index("gene_name")
deg_down = deg.sort_values(f"{column}", ascending=True).dropna(axis=0).set_index("gene_name")
up_genes = list(deg_up.iloc[:600].index.values)
down_genes = list(deg_down.iloc[:600].index.values)

METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

input_gene_set = {
    "up_genes": up_genes,
    "down_genes": down_genes
}

all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": all_genes
            }
        },
        "fields": ["id", "meta.symbol"]
    }
}
res = requests.post(METADATA_API + "entities/find", json=payload)
entities = res.json()

for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

for e in entities:
    symbol = e["meta"]["symbol"]
    if symbol in input_gene_set["up_genes"]:
        for_enrichment["up_entities"].append(e["id"])
    elif symbol in input_gene_set["down_genes"]:
        for_enrichment["down_entities"].append(e["id"])
for_enrichment['up_entities'] = for_enrichment['up_entities'][:500]
for_enrichment['down_entities'] = for_enrichment['down_entities'][:500]
print(len(for_enrichment['up_entities']), len(for_enrichment['down_entities']))

payload = {
    "meta": {
        "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
        **for_enrichment
    },
    "type": "signature"
}
res = requests.post(METADATA_API + "user_input", json=payload)
persistent_id = res.json()["id"]
print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

NUM_CASE = 120
query = {
    **for_enrichment,
    "limit": NUM_CASE,
    "database": signature
}
#
res = requests.post(DATA_API + "enrich/ranktwosided", json=query)
results = res.json()

# # Optional, multiply z-down and direction-down with -1
# for i in results["results"]:
#     i["z-down"] = -i["z-down"]
#     i["direction-down"] = -i["direction-down"]

# results = results['results'][:NUM_CASE]
# sigids = {i['uuid']: i for i in results}

# payload = {
#     "filter": {
#         "where": {
#             "id": {
#                 "inq": list(sigids.keys())
#             }
#         }
#     }
# }

# res = requests.post(METADATA_API + "signatures/find", json=payload)
# signatures = res.json()

# ## Merge the scores and the metadata
# for sig in signatures:
#     uid = sig["id"]
#     scores = sigids[uid]
#     # scores.pop("uuid")
#     sig["scores"] = scores

# for i in signatures:
#     dict1 = i['meta']
#     dict2 = i['scores']

#     dict1.update(dict2)

# df_mimickers = pd.DataFrame([x['meta'] for x in signatures])
# file_name = f'Norman_K562_{CONDITION}'

# df_mimickers = df_mimickers.sort_values(by=['z-sum'], ascending=False)
# df_mimickers.index = [f'Top-{x}' for x in range(1, len(df_mimickers)+1)]
# df_mimickers.dropna(subset=['pert_name'], inplace=True)
# # df_mimickers.dropna(subset=['pert_name'], ).resbet_index(drop=True, )
# # df_mimickers.dropna(subset=['pert_name'], inplace=True)
# # df_mimickers['z-sum']
# mimickers_save_path = f"{path}/{file_name}.csv"
# df_mimickers.to_csv(mimickers_save_path)

# df_mimickers_top10 = df_mimickers.iloc[:10]
# df_mimickers_top30 = df_mimickers.iloc[:30]
# df_mimickers_top50 = df_mimickers.iloc[:50]
# df_mimickers_top100 = df_mimickers.iloc[:100]

# benchmark = {}

# for i, rank in zip([df_mimickers_top10, df_mimickers_top30, df_mimickers_top50, df_mimickers_top100], [10, 30, 50, 100]):
#     pert_list = i.pert_name.unique()

#     if CONDITION not in benchmark.keys():
#         benchmark[CONDITION] = {}
#     if len(CONDITION.split('_')) > 1:

#         CONDITION_LIST = CONDITION.split('_')
#         print(CONDITION_LIST)
#         if len(set(CONDITION_LIST).intersection(pert_list)) > 0:
#             print(f'TOP-{rank} Accuracy: True')
#             benchmark[CONDITION][f'Top-{rank}'] = 1
#         else:   
#             print(f'Top-{rank} Accuracy: False')
#             benchmark[CONDITION][f'Top-{rank}'] = 0
#     else:
#         if CONDITION in pert_list:
            
#             print(f'TOP-{rank} Accuracy: True')
#             benchmark[CONDITION][f'Top-{rank}'] = 1
#         else: 
#             print(f'Top-{rank} Accuracy: False')
#             benchmark[CONDITION][f'Top-{rank}'] = 0
# new_benchmark_df = pd.DataFrame(benchmark)
# display(df_mimickers.head())

# if signature == 'l1000_xpr':


#     benchmark_df = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_xpr_benchmark_Norman.csv", index_col=0)
#     if CONDITION not in benchmark_df.columns:
#         benchmark_df = pd.concat([benchmark_df, new_benchmark_df], axis=1)
#         benchmark_df.to_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_xpr_benchmark_Norman.csv", index=True)
#     else:
#         print('already exists')
# elif signature == "LINCS consensus gene (CGS) knockdown signatures":
#     benchmark_df = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_CGS_benchmark_Norman.csv", index_col=0)
#     if CONDITION not in benchmark_df.columns:
#         benchmark_df = pd.concat([benchmark_df, new_benchmark_df], axis=1)
#         benchmark_df.to_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_CGS_benchmark_Norman.csv", index=True)
#     else:
#         print('already exists')
# return benchmark_df

500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/01cab592-416b-57e8-84e7-7f901ce248b0


In [110]:
CONDITION = 'TGFBR2_IGDCC3'
deg = pd.read_csv(f"{CONDITION}.csv")
try: 
    deg = deg.drop(['Unnamed: 0'], axis=1)
except:
    pass
column = deg.columns[1]
deg_up = deg.sort_values(f"{column}", ascending=False).dropna(axis=0).set_index("gene_name")
deg_down = deg.sort_values(f"{column}", ascending=True).dropna(axis=0).set_index("gene_name")
up_genes = list(deg_up.iloc[:600].index.values)
down_genes = list(deg_down.iloc[:600].index.values)

METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

In [111]:
input_gene_set = {
    "up_genes": up_genes,
    "down_genes": down_genes
}

all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": all_genes
            }
        },
        "fields": ["id", "meta.symbol"]
    }
}
res = requests.post(METADATA_API + "entities/find", json=payload)
entities = res.json()

In [112]:
for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

for e in entities:
    symbol = e["meta"]["symbol"]
    if symbol in input_gene_set["up_genes"]:
        for_enrichment["up_entities"].append(e["id"])
    elif symbol in input_gene_set["down_genes"]:
        for_enrichment["down_entities"].append(e["id"])
for_enrichment['up_entities'] = for_enrichment['up_entities'][:500]
for_enrichment['down_entities'] = for_enrichment['down_entities'][:500]
print(len(for_enrichment['up_entities']), len(for_enrichment['down_entities']))

500 500


In [114]:
CONDITION = 'TGFBR2_IGDCC3'
deg = pd.read_csv(f"{CONDITION}.csv")
try: 
    deg = deg.drop(['Unnamed: 0'], axis=1)
except:
    pass
column = deg.columns[1]
deg_up = deg.sort_values(f"{column}", ascending=False).dropna(axis=0).set_index("gene_name")
deg_down = deg.sort_values(f"{column}", ascending=True).dropna(axis=0).set_index("gene_name")
up_genes = list(deg_up.iloc[:600].index.values)
down_genes = list(deg_down.iloc[:600].index.values)

METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

input_gene_set = {
    "up_genes": up_genes,
    "down_genes": down_genes
}

all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": all_genes
            }
        },
        "fields": ["id", "meta.symbol"]
    }
}
res = requests.post(METADATA_API + "entities/find", json=payload)
entities = res.json()

for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

for e in entities:
    symbol = e["meta"]["symbol"]
    if symbol in input_gene_set["up_genes"]:
        for_enrichment["up_entities"].append(e["id"])
    elif symbol in input_gene_set["down_genes"]:
        for_enrichment["down_entities"].append(e["id"])
for_enrichment['up_entities'] = for_enrichment['up_entities'][:500]
for_enrichment['down_entities'] = for_enrichment['down_entities'][:500]
print(len(for_enrichment['up_entities']), len(for_enrichment['down_entities']))

payload = {
    "meta": {
        "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
        **for_enrichment
    },
    "type": "signature"
}
res = requests.post(METADATA_API + "user_input", json=payload)
persistent_id = res.json()["id"]
print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/01cab592-416b-57e8-84e7-7f901ce248b0


In [122]:
NUM_CASE = 120
query = {
    **for_enrichment,
    "limit": NUM_CASE,
    "database": "LINCS consensus gene (CGS) knockdown signatures"
}

res = requests.post(DATA_API + "enrich/ranktwosided", json=query)
results = res.json()

# Optional, multiply z-down and direction-down with -1
for i in results["results"]:
    i["z-down"] = -i["z-down"]
    i["direction-down"] = -i["direction-down"]

In [123]:
results

{'queryTimeSec': 0.235,
 'reversers': 12877,
 'results': [{'direction-up': 1,
   'fdr-down': 0.05322683568975828,
   'logp-avg': 6.783702136484638,
   'logp-fisher': 18.597417823586508,
   'p-up': 3.707235288974431e-06,
   'type': 'mimickers',
   'uuid': '004e1085-775d-438b-b18e-69e94aefd994',
   'z-down': -3.0537096948679805,
   'fdr-up': 0.1381871953965219,
   'z-sum': 7.681085791541299,
   'z-up': 4.627376096673318,
   'p-down': 0.002260444826207575,
   'p-down-bonferroni': 1,
   'rank': 0,
   'p-up-bonferroni': 0.1381871953965219,
   'direction-down': 1},
  {'direction-up': 1,
   'fdr-down': 0.04291728515961618,
   'logp-avg': 7.23093101452656,
   'logp-fisher': 15.255523931458951,
   'p-up': 0.00018809542237385735,
   'type': 'mimickers',
   'uuid': 'b6863d5a-6419-4b73-83ac-93228abf6e05',
   'z-down': -3.2250578494476856,
   'fdr-up': 0.9055608536961262,
   'z-sum': 6.959624172760186,
   'z-up': 3.7345663233125013,
   'p-down': 0.001259597852840244,
   'p-down-bonferroni': 1,
   '

In [124]:
results = results['results'][:NUM_CASE]
results

[{'direction-up': 1,
  'fdr-down': 0.05322683568975828,
  'logp-avg': 6.783702136484638,
  'logp-fisher': 18.597417823586508,
  'p-up': 3.707235288974431e-06,
  'type': 'mimickers',
  'uuid': '004e1085-775d-438b-b18e-69e94aefd994',
  'z-down': -3.0537096948679805,
  'fdr-up': 0.1381871953965219,
  'z-sum': 7.681085791541299,
  'z-up': 4.627376096673318,
  'p-down': 0.002260444826207575,
  'p-down-bonferroni': 1,
  'rank': 0,
  'p-up-bonferroni': 0.1381871953965219,
  'direction-down': 1},
 {'direction-up': 1,
  'fdr-down': 0.04291728515961618,
  'logp-avg': 7.23093101452656,
  'logp-fisher': 15.255523931458951,
  'p-up': 0.00018809542237385735,
  'type': 'mimickers',
  'uuid': 'b6863d5a-6419-4b73-83ac-93228abf6e05',
  'z-down': -3.2250578494476856,
  'fdr-up': 0.9055608536961262,
  'z-sum': 6.959624172760186,
  'z-up': 3.7345663233125013,
  'p-down': 0.001259597852840244,
  'p-down-bonferroni': 1,
  'rank': 1,
  'p-up-bonferroni': 1,
  'direction-down': 1},
 {'direction-up': 1,
  'fdr-

In [121]:



results = results['results'][:NUM_CASE]
sigids = {i['uuid']: i for i in results}

payload = {
    "filter": {
        "where": {
            "id": {
                "inq": list(sigids.keys())
            }
        }
    }
}

res = requests.post(METADATA_API + "signatures/find", json=payload)
signatures = res.json()

## Merge the scores and the metadata
for sig in signatures:
    uid = sig["id"]
    scores = sigids[uid]
    # scores.pop("uuid")
    sig["scores"] = scores

for i in signatures:
    dict1 = i['meta']
    dict2 = i['scores']

    dict1.update(dict2)

df_mimickers = pd.DataFrame([x['meta'] for x in signatures])

In [119]:
df_mimickers

""


In [54]:
# CONDITION = 'TGFBR2_IGDCC3'
# deg = pd.read_csv(f"{CONDITION}.csv")
# try: 
#     deg = deg.drop(['Unnamed: 0'], axis=1)
# except:
#     pass
# column = deg.columns[1]
# deg_up = deg.sort_values(f"{column}", ascending=False).dropna(axis=0).set_index("gene_name")
# deg_down = deg.sort_values(f"{column}", ascending=True).dropna(axis=0).set_index("gene_name")
# up_genes = list(deg_up.iloc[:600].index.values)
# down_genes = list(deg_down.iloc[:600].index.values)

# METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
# DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

# input_gene_set = {
#     "up_genes": up_genes,
#     "down_genes": down_genes
# }

# all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

# payload = {
#     "filter": {
#         "where": {
#             "meta.symbol": {
#                 "inq": all_genes
#             }
#         },
#         "fields": ["id", "meta.symbol"]
#     }
# }
# res = requests.post(METADATA_API + "entities/find", json=payload)
# entities = res.json()

# for_enrichment = {
#     "up_entities": [],
#     "down_entities": []
# }

# for e in entities:
#     symbol = e["meta"]["symbol"]
#     if symbol in input_gene_set["up_genes"]:
#         for_enrichment["up_entities"].append(e["id"])
#     elif symbol in input_gene_set["down_genes"]:
#         for_enrichment["down_entities"].append(e["id"])
# for_enrichment['up_entities'] = for_enrichment['up_entities'][:500]
# for_enrichment['down_entities'] = for_enrichment['down_entities'][:500]
# print(len(for_enrichment['up_entities']), len(for_enrichment['down_entities']))

# payload = {
#     "meta": {
#         "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
#         **for_enrichment
#     },
#     "type": "signature"
# }
# res = requests.post(METADATA_API + "user_input", json=payload)
# persistent_id = res.json()["id"]
# print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

# NUM_CASE = 120
# query = {
#     **for_enrichment,
#     "limit": NUM_CASE,
#     "database": "l1000_xpr"
# }

# res = requests.post(DATA_API + "enrich/ranktwosided", json=query)
# results = res.json()

# # Optional, multiply z-down and direction-down with -1
# for i in results["results"]:
#     i["z-down"] = -i["z-down"]
#     i["direction-down"] = -i["direction-down"]

# results = results['results'][:NUM_CASE]
# sigids = {i['uuid']: i for i in results}

# payload = {
#     "filter": {
#         "where": {
#             "id": {
#                 "inq": list(sigids.keys())
#             }
#         }
#     }
# }

# res = requests.post(METADATA_API + "signatures/find", json=payload)
# signatures = res.json()

# ## Merge the scores and the metadata
# for sig in signatures:
#     uid = sig["id"]
#     scores = sigids[uid]
#     # scores.pop("uuid")
#     sig["scores"] = scores

# for i in signatures:
#     dict1 = i['meta']
#     dict2 = i['scores']

#     dict1.update(dict2)

# df_mimickers = pd.DataFrame([x['meta'] for x in signatures])
# file_name = f'Norman_K562_{CONDITION}'

# df_mimickers = df_mimickers.sort_values(by=['z-sum'], ascending=False)
# df_mimickers.index = [f'Top-{x}' for x in range(1, len(df_mimickers)+1)]
# df_mimickers.dropna(subset=['pert_name'], inplace=True)
# # df_mimickers.dropna(subset=['pert_name'], ).resbet_index(drop=True, )
# # df_mimickers.dropna(subset=['pert_name'], inplace=True)
# # df_mimickers['z-sum']
# mimickers_save_path = f"{path}/{file_name}.csv"
# df_mimickers.to_csv(mimickers_save_path)

# df_mimickers_top10 = df_mimickers.iloc[:10]
# df_mimickers_top30 = df_mimickers.iloc[:30]
# df_mimickers_top50 = df_mimickers.iloc[:50]
# df_mimickers_top100 = df_mimickers.iloc[:100]

# benchmark = {}

# for i, rank in zip([df_mimickers_top10, df_mimickers_top30, df_mimickers_top50, df_mimickers_top100], [10, 30, 50, 100]):
#     pert_list = i.pert_name.unique()

#     if CONDITION not in benchmark.keys():
#         benchmark[CONDITION] = {}
#     if len(CONDITION.split('_')) > 1:

#         CONDITION_LIST = CONDITION.split('_')
#         print(CONDITION_LIST)
#         if len(set(CONDITION_LIST).intersection(pert_list)) > 0:
#             print(f'TOP-{rank} Accuracy: True')
#             benchmark[CONDITION][f'Top-{rank}'] = 1
#         else:   
#             print(f'Top-{rank} Accuracy: False')
#             benchmark[CONDITION][f'Top-{rank}'] = 0
#     else:
#         if CONDITION in pert_list:
            
#             print(f'TOP-{rank} Accuracy: True')
#             benchmark[CONDITION][f'Top-{rank}'] = 1
#         else: 
#             print(f'Top-{rank} Accuracy: False')
#             benchmark[CONDITION][f'Top-{rank}'] = 0
# new_benchmark_df = pd.DataFrame(benchmark)
# display(df_mimickers.head())
# benchmark_df = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_xpr_benchmark_Norman.csv", index_col=0)
# if CONDITION not in benchmark_df.columns:
#     benchmark_df = pd.concat([benchmark_df, new_benchmark_df], axis=1)
#     benchmark_df.to_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_xpr_benchmark_Norman.csv", index=True)
# else:
#     print('already exists')

500 500


Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/01cab592-416b-57e8-84e7-7f901ce248b0
['TGFBR2', 'IGDCC3']
TOP-10 Accuracy: True
['TGFBR2', 'IGDCC3']
TOP-30 Accuracy: True
['TGFBR2', 'IGDCC3']
TOP-50 Accuracy: True
['TGFBR2', 'IGDCC3']
TOP-100 Accuracy: True


,md5,doid,sha256,tissue,anatomy,cmap_id,disease,version,filename,local_id,...,z-down,fdr-up,z-sum,z-up,p-down,p-down-bonferroni,rank,p-up-bonferroni,direction-down,pert_dose
Top-1,101e0a4c920c46933113ea7d4ac3fe15,DOID:1909,16baa8ad081bffc64ff9f69d6f81420c48c4c9994bcf0f...,skin of body,UBERON:0002097,XPR019_A375.311_96H:P24,melanoma,1,L1000_LINCS_DCIC_XPR019_A375.311_96H_P24_SMG1.tsv,XPR019_A375.311_96H_P24_SMG1,...,-12.996620,5.976311e-05,17.847755,4.851135,0.000000e+00,6.946711e-319,0,1.728349e-01,1,NaN
Top-2,e0a319a29d32b8c4decb371b9aba0c96,DOID:305,c01c5c0f2cceb442abcd8e2a80fa68bb8639e8e60d3763...,hypopharynx,UBERON:0001051,XPR017_BICR6.311_96H:A21,carcinoma,1,L1000_LINCS_DCIC_XPR017_BICR6.311_96H_A21_GRB2...,XPR017_BICR6.311_96H_A21_GRB2,...,-6.787281,1.157793e-319,17.382710,10.595429,1.149303e-11,1.615954e-06,1,6.946711e-319,1,NaN
Top-3,77c819d6800cd226f2d6dcffa814d77f,DOID:3459,72e3896c1cbaab811fe5f48d5ac40af2067f344c27838a...,breast,UBERON:0000310,XPR036_MCF7.311_96H:H18,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR036_MCF7.311_96H_H18_ASNS.tsv,XPR036_MCF7.311_96H_H18_ASNS,...,-10.454484,2.002795e-08,17.108720,6.654236,0.000000e+00,6.946711e-319,2,4.025618e-06,1,NaN
Top-4,636f826ae3145d6217c61f3d7fb939c2,DOID:1909,0f2ea9ee6cf2787f5a71588cfde233dc3fb423c4177766...,skin of body,UBERON:0002097,XPR036_A375.311_96H:N15,melanoma,1,L1000_LINCS_DCIC_XPR036_A375.311_96H_N15_RPS6K...,XPR036_A375.311_96H_N15_RPS6KA4,...,-10.605589,1.292897e-07,16.872449,6.266859,0.000000e+00,6.946711e-319,3,5.201984e-05,1,NaN
Top-5,0cc7103130b8ec32fb26ac0ae5aadc31,DOID:1909,f31baab1db29d567a2a5a817780bb866f883d8ba2662bb...,skin of body,UBERON:0002097,XPR019_A375.311_96H:F04,melanoma,1,L1000_LINCS_DCIC_XPR019_A375.311_96H_F04_SMG1.tsv,XPR019_A375.311_96H_F04_SMG1,...,-9.550904,3.211019e-09,16.549430,6.998525,0.000000e+00,6.946711e-319,4,3.660561e-07,1,NaN


In [69]:
result = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_xpr_benchmark_Norman.csv", index_col=0)
result.columns.tolist()

['SNAI1',
 'PTPN12_OSR2',
 'TGFBR2_C19orf26',
 'CNN1_MAPK1',
 'SET',
 'S1PR2',
 'CEBPE',
 'MAP2K3',
 'BAK1',
 'KMT2A',
 'ETS2_MAPK1',
 'OSR2',
 'CEBPE_PTPN12',
 'SLC4A1',
 'SPI1',
 'GLB1L2',
 'CEBPE_RUNX1T1',
 'TSC22D1',
 'UBASH3B_CNN1',
 'FOXA1',
 'LYL1_CEBPB',
 'COL2A1',
 'SAMD1_ZBTB1',
 'LHX1_ELMSAN1',
 'AHR_FEV',
 'FEV_CBFA2T3',
 'RUNX1T1',
 'MAP2K6',
 'ETS2',
 'CEBPB',
 'KLF1_MAP2K6',
 'CBL_CNN1',
 'KLF1',
 'ZC3HAV1_HOXC13',
 'TMSB4X_BAK1',
 'MAP2K3_ELMSAN1',
 'UBASH3B_PTPN9',
 'CBL',
 'CBL_TGFBR2',
 'SLC6A9',
 'SET_KLF1',
 'FOXF1_HOXB9',
 'TGFBR2_IGDCC3',
 'ETS2_IGDCC3',
 'ARRDC3',
 'STIL']

In [76]:
result_to_sigs = pd.DataFrame(columns=["LINCS consensus gene (CGS) knockdown signatures"], index=result.index)
result_to_sigs.to_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/sigcom_CGS_benchmark_Norman.csv", index=True)

In [84]:
CONDITION = condition_list[4]

In [102]:
CONDITION

'TGFBR2_IGDCC3'

In [103]:
deg_analysis(CONDITION, 'CGS')

500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/01cab592-416b-57e8-84e7-7f901ce248b0


KeyError: 'z-sum'

In [104]:
results

[{'direction-up': 1,
  'fdr-down': 0.05322683568975828,
  'logp-avg': 6.783702136484638,
  'logp-fisher': 18.597417823586508,
  'p-up': 3.707235288974431e-06,
  'type': 'mimickers',
  'uuid': '004e1085-775d-438b-b18e-69e94aefd994',
  'z-down': -3.0537096948679805,
  'fdr-up': 0.1381871953965219,
  'z-sum': 7.681085791541299,
  'z-up': 4.627376096673318,
  'p-down': 0.002260444826207575,
  'p-down-bonferroni': 1,
  'rank': 0,
  'p-up-bonferroni': 0.1381871953965219,
  'direction-down': 1},
 {'direction-up': 1,
  'fdr-down': 0.04291728515961618,
  'logp-avg': 7.23093101452656,
  'logp-fisher': 15.255523931458951,
  'p-up': 0.00018809542237385735,
  'type': 'mimickers',
  'uuid': 'b6863d5a-6419-4b73-83ac-93228abf6e05',
  'z-down': -3.2250578494476856,
  'fdr-up': 0.9055608536961262,
  'z-sum': 6.959624172760186,
  'z-up': 3.7345663233125013,
  'p-down': 0.001259597852840244,
  'p-down-bonferroni': 1,
  'rank': 1,
  'p-up-bonferroni': 1,
  'direction-down': 1},
 {'direction-up': 1,
  'fdr-

In [93]:
for i in results["results"]:
    i["z-down"] = -i["z-down"]
    i["direction-down"] = -i["direction-down"]

In [97]:
sigids

{'004e1085-775d-438b-b18e-69e94aefd994': {'direction-up': 1,
  'fdr-down': 0.05322683568975828,
  'logp-avg': 6.783702136484638,
  'logp-fisher': 18.597417823586508,
  'p-up': 3.707235288974431e-06,
  'type': 'mimickers',
  'uuid': '004e1085-775d-438b-b18e-69e94aefd994',
  'z-down': -3.0537096948679805,
  'fdr-up': 0.1381871953965219,
  'z-sum': 7.681085791541299,
  'z-up': 4.627376096673318,
  'p-down': 0.002260444826207575,
  'p-down-bonferroni': 1,
  'rank': 0,
  'p-up-bonferroni': 0.1381871953965219,
  'direction-down': 1},
 'b6863d5a-6419-4b73-83ac-93228abf6e05': {'direction-up': 1,
  'fdr-down': 0.04291728515961618,
  'logp-avg': 7.23093101452656,
  'logp-fisher': 15.255523931458951,
  'p-up': 0.00018809542237385735,
  'type': 'mimickers',
  'uuid': 'b6863d5a-6419-4b73-83ac-93228abf6e05',
  'z-down': -3.2250578494476856,
  'fdr-up': 0.9055608536961262,
  'z-sum': 6.959624172760186,
  'z-up': 3.7345663233125013,
  'p-down': 0.001259597852840244,
  'p-down-bonferroni': 1,
  'rank':

In [94]:
results = results['results'][:NUM_CASE]
sigids = {i['uuid']: i for i in results}

payload = {
    "filter": {
        "where": {
            "id": {
                "inq": list(sigids.keys())
            }
        }
    }
}

res = requests.post(METADATA_API + "signatures/find", json=payload)
signatures = res.json()

In [100]:
sigids

{'004e1085-775d-438b-b18e-69e94aefd994': {'direction-up': 1,
  'fdr-down': 0.05322683568975828,
  'logp-avg': 6.783702136484638,
  'logp-fisher': 18.597417823586508,
  'p-up': 3.707235288974431e-06,
  'type': 'mimickers',
  'uuid': '004e1085-775d-438b-b18e-69e94aefd994',
  'z-down': -3.0537096948679805,
  'fdr-up': 0.1381871953965219,
  'z-sum': 7.681085791541299,
  'z-up': 4.627376096673318,
  'p-down': 0.002260444826207575,
  'p-down-bonferroni': 1,
  'rank': 0,
  'p-up-bonferroni': 0.1381871953965219,
  'direction-down': 1},
 'b6863d5a-6419-4b73-83ac-93228abf6e05': {'direction-up': 1,
  'fdr-down': 0.04291728515961618,
  'logp-avg': 7.23093101452656,
  'logp-fisher': 15.255523931458951,
  'p-up': 0.00018809542237385735,
  'type': 'mimickers',
  'uuid': 'b6863d5a-6419-4b73-83ac-93228abf6e05',
  'z-down': -3.2250578494476856,
  'fdr-up': 0.9055608536961262,
  'z-sum': 6.959624172760186,
  'z-up': 3.7345663233125013,
  'p-down': 0.001259597852840244,
  'p-down-bonferroni': 1,
  'rank':

In [99]:
payload

{'filter': {'where': {'id': {'inq': ['004e1085-775d-438b-b18e-69e94aefd994',
     'b6863d5a-6419-4b73-83ac-93228abf6e05',
     '6bc9769b-e2b2-4bb1-b282-4ba7ef4cec6c',
     '9d5ea19c-680a-4418-9b8e-02e3b4bfc21a',
     '53146c24-cce1-4345-8317-251856c064ac',
     '448702d8-b253-4410-b676-a525781b53aa',
     '6fe0232a-b4a6-411f-bdc6-d9b21084eac1',
     '11958533-72f7-4831-8677-b4cd339ef031',
     '2e540f0c-fa4b-412e-ab54-2651d72eff3c',
     '0a05a3c5-4b1c-4eb8-8d12-a6cc508dea1c',
     '307152f0-1342-45e6-9f73-7916ad958a36',
     'a06cbe8c-5a79-4b1e-ae97-efe0e63ee2c6',
     '836284cc-a586-47ce-a169-b7e42aafe930',
     '3fbc258b-e3bb-4b46-af87-6696b128c09f',
     '5f3693a0-0de6-4ebe-b508-171083802d44',
     '24b44e60-6ac0-42a3-96ef-baa5e12debe7',
     '921f311c-b855-493d-8fba-d80c78246b29',
     '1dafcc6d-734c-4d67-94ae-687aaf2b2bfc',
     'e35696a6-2e54-4539-bb78-c67a85cc8c7b',
     '6e9a11bb-cb20-446e-8fd2-a7f421f687f1',
     '2e73a7bc-aeaa-4ec4-8009-91d305b15623',
     'e37d8a3c-0ebc-420

In [96]:
signatures

[]

In [65]:
condition_list = [x.split('.')[0] for x in os.listdir(deg_result_path) if 'csv' in x]
condition_list = [x for x in condition_list if x not in result]

for CONDITION in condition_list:
    deg_analysis(CONDITION)

500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/7125464d-fb0f-5e71-84dd-77fc603d1641
Top-10 Accuracy: False
Top-30 Accuracy: False
Top-50 Accuracy: False
Top-100 Accuracy: False


,md5,doid,sha256,tissue,anatomy,cmap_id,disease,version,filename,local_id,...,uuid,z-down,fdr-up,z-sum,z-up,p-down,p-down-bonferroni,rank,p-up-bonferroni,direction-down
Top-1,59ea7c564d09efa8a0d18bf3f8460c9e,DOID:3069,8c877409c997d04ef66ac7008ad86db5c87b914cd59225...,brain,UBERON:0000955,XPR030_U251MG.311_96H:N08,astrocytoma,1,L1000_LINCS_DCIC_XPR030_U251MG.311_96H_N08_HSP...,XPR030_U251MG.311_96H_N08_HSPA5,...,75315367-6aae-597c-b595-fd40d26f0f2b,-7.109059,3.659912e-06,12.830486,5.721427,1.176392e-12,1.654043e-07,0,0.001490,1
Top-2,fe1b6de74687c7ba2ee2b697207d3a2b,DOID:3717,0e5dd3909782bb45ea4c0e1e0a4895a960ce14aa8c3c60...,stomach,UBERON:0000945,XPR030_AGS.311_96H:A08,gastric adenocarcinoma,1,L1000_LINCS_DCIC_XPR030_AGS.311_96H_A08_PSMB1.tsv,XPR030_AGS.311_96H_A08_PSMB1,...,1f283221-6ecd-5761-b795-83c99d7731a9,-7.218587,7.743462e-05,12.168173,4.949586,5.291323e-13,7.439759e-08,1,0.104726,1
Top-3,626bc938c0a7dcaa91ac1ef2590ffddf,DOID:305,97cdfdc3126753c82d36277f318a1a4b7ad5854fa05d4b...,hypopharynx,UBERON:0001051,XPR019_BICR6.311_96H:L07,carcinoma,1,L1000_LINCS_DCIC_XPR019_BICR6.311_96H_L07_LGAL...,XPR019_BICR6.311_96H_L07_LGALS3,...,25eca061-29cb-5657-bc20-a1cad6fec1fe,-5.341781,2.544947e-07,11.691728,6.349947,9.223679e-08,1.296877e-02,2,0.000030,1
Top-4,ca0476c9e823aa40a27e3279534164eb,DOID:3459,7d1e2bc892a1e7927815ff8f7142d68a103035c03fb016...,breast,UBERON:0000310,XPR027_MCF7.311_96H:G23,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR027_MCF7.311_96H_G23_VMP1.tsv,XPR027_MCF7.311_96H_G23_VMP1,...,a3589579-474e-5a3c-913a-ac16faca704e,-5.418942,4.727066e-07,11.624113,6.205170,6.009097e-08,8.448971e-03,3,0.000077,1
Top-5,101e0a4c920c46933113ea7d4ac3fe15,DOID:1909,16baa8ad081bffc64ff9f69d6f81420c48c4c9994bcf0f...,skin of body,UBERON:0002097,XPR019_A375.311_96H:P24,melanoma,1,L1000_LINCS_DCIC_XPR019_A375.311_96H_P24_SMG1.tsv,XPR019_A375.311_96H_P24_SMG1,...,658f0ea5-a51a-508f-8250-682f4ece0f18,-7.743045,2.483950e-03,11.601626,3.858581,9.769963e-15,1.373686e-09,4,1.000000,1


500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/2cb1c223-abf7-5bd2-ae23-f3a911fa4a3f
Top-10 Accuracy: False
Top-30 Accuracy: False
Top-50 Accuracy: False
Top-100 Accuracy: False


,md5,doid,sha256,tissue,anatomy,cmap_id,disease,version,filename,local_id,...,uuid,z-down,fdr-up,z-sum,z-up,p-down,p-down-bonferroni,rank,p-up-bonferroni,direction-down
Top-1,636f826ae3145d6217c61f3d7fb939c2,DOID:1909,0f2ea9ee6cf2787f5a71588cfde233dc3fb423c4177766...,skin of body,UBERON:0002097,XPR036_A375.311_96H:N15,melanoma,1,L1000_LINCS_DCIC_XPR036_A375.311_96H_N15_RPS6K...,XPR036_A375.311_96H_N15_RPS6KA4,...,d56ce067-db5b-5f00-9309-86cce78e1df6,-12.629951,1.280026e-09,19.749220,7.119269,0.0,6.946711e-319,0,1.536031e-07,1
Top-2,77c819d6800cd226f2d6dcffa814d77f,DOID:3459,72e3896c1cbaab811fe5f48d5ac40af2067f344c27838a...,breast,UBERON:0000310,XPR036_MCF7.311_96H:H18,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR036_MCF7.311_96H_H18_ASNS.tsv,XPR036_MCF7.311_96H_H18_ASNS,...,8cf492bf-5378-5cb0-8b6b-ed104507718a,-12.654605,1.223746e-08,19.365164,6.710558,0.0,6.946711e-319,1,2.741191e-06,1
Top-3,08b2def97b6ae8bcc3df849f0b9ea5aa,DOID:3459,8cbd0abbddd6362451200366f914fa1410b065a6e961ea...,breast,UBERON:0000310,XPR027_MCF7.311_96H:F24,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR027_MCF7.311_96H_F24_CNGA3...,XPR027_MCF7.311_96H_F24_CNGA3,...,ba9421ec-60f1-57cb-bc47-7c8a3cfece6d,-12.745294,2.095503e-08,19.350916,6.605621,0.0,6.946711e-319,2,5.594992e-06,1
Top-4,f24338d3c51107fdcfc2a6cf2e9e4a2d,DOID:2526,1dde1980e42754b9e12b851f45b693f4051773bfe045b2...,prostate gland,UBERON:0002367,XPR018_PC3.311B_96H:B24,prostate adenocarcinoma,1,L1000_LINCS_DCIC_XPR018_PC3.311B_96H_B24_POLE2...,XPR018_PC3.311B_96H_B24_POLE2,...,4eb5eacb-fb93-5881-8b78-5d34492930e6,-12.004785,8.435219e-10,19.195733,7.190948,0.0,6.946711e-319,3,9.110036e-08,1
Top-5,14166e00cb1900a71eba563f53d5e788,DOID:3459,d159fb03a119165252d162ff0ec6205bc2437f96d202c2...,breast,UBERON:0000310,XPR020_MCF7.311_96H:L23,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR020_MCF7.311_96H_L23_MILR1...,XPR020_MCF7.311_96H_L23_MILR1,...,14210764-3f71-5a99-804f-9c2f0e711461,-11.612719,1.456940e-10,19.146306,7.533587,0.0,6.946711e-319,4,6.993311e-09,1


500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/d26a19f4-1765-5b1a-853d-0dfe0bb8d268
Top-10 Accuracy: False
Top-30 Accuracy: False
Top-50 Accuracy: False
Top-100 Accuracy: False


...

,md5,doid,sha256,tissue,anatomy,cmap_id,disease,version,filename,local_id,...,uuid,z-down,fdr-up,z-sum,z-up,p-down,p-down-bonferroni,rank,p-up-bonferroni,direction-down
Top-1,636f826ae3145d6217c61f3d7fb939c2,DOID:1909,0f2ea9ee6cf2787f5a71588cfde233dc3fb423c4177766...,skin of body,UBERON:0002097,XPR036_A375.311_96H:N15,melanoma,1,L1000_LINCS_DCIC_XPR036_A375.311_96H_N15_RPS6K...,XPR036_A375.311_96H_N15_RPS6KA4,...,d56ce067-db5b-5f00-9309-86cce78e1df6,-11.211257,4.048670e-09,18.042766,6.831509,0.000000e+00,6.946711e-319,0,1.188457e-06,1
Top-2,77c819d6800cd226f2d6dcffa814d77f,DOID:3459,72e3896c1cbaab811fe5f48d5ac40af2067f344c27838a...,breast,UBERON:0000310,XPR036_MCF7.311_96H:H18,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR036_MCF7.311_96H_H18_ASNS.tsv,XPR036_MCF7.311_96H_H18_ASNS,...,8cf492bf-5378-5cb0-8b6b-ed104507718a,-11.841359,2.845521e-07,17.837986,5.996627,0.000000e+00,6.946711e-319,1,2.842676e-04,1
Top-3,e0a319a29d32b8c4decb371b9aba0c96,DOID:305,c01c5c0f2cceb442abcd8e2a80fa68bb8639e8e60d3763...,hypopharynx,UBERON:0001051,XPR017_BICR6.311_96H:A21,carcinoma,1,L1000_LINCS_DCIC_XPR017_BICR6.311_96H_A21_GRB2...,XPR017_BICR6.311_96H_A21_GRB2,...,c845ff19-094b-5be5-9986-ba96b239e40e,-7.682735,2.573094e-320,17.402637,9.719902,1.576517e-14,2.216630e-09,2,6.946711e-319,1
Top-4,442795e5c580673fb1c04a0dcaf1698e,DOID:3459,b5ce9ed5ff6b4ab98c75583d744a1d377b1bcecddfafe6...,breast,UBERON:0000310,XPR017_MCF7.311_96H:K24,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR017_MCF7.311_96H_K24_HIST1...,XPR017_MCF7.311_96H_K24_HIST1H2AH,...,37e903f4-fa3d-5e02-b9eb-16bb2d8cd223,-8.986860,2.542211e-11,16.697810,7.710950,0.000000e+00,6.946711e-319,3,1.779548e-09,1
Top-5,101e0a4c920c46933113ea7d4ac3fe15,DOID:1909,16baa8ad081bffc64ff9f69d6f81420c48c4c9994bcf0f...,skin of body,UBERON:0002097,XPR019_A375.311_96H:P24,melanoma,1,L1000_LINCS_DCIC_XPR019_A375.311_96H_P24_SMG1.tsv,XPR019_A375.311_96H_P24_SMG1,...,658f0ea5-a51a-508f-8250-682f4ece0f18,-11.431128,9.660675e-06,16.640503,5.209375,0.000000e+00,6.946711e-319,4,2.669245e-02,1


500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/6fe03b9a-c144-540a-afa2-cd20bdca7cce
Top-10 Accuracy: False
Top-30 Accuracy: False
Top-50 Accuracy: False
Top-100 Accuracy: False


,md5,doid,sha256,tissue,anatomy,cmap_id,disease,version,filename,local_id,...,z-down,fdr-up,z-sum,z-up,p-down,p-down-bonferroni,rank,p-up-bonferroni,direction-down,pert_dose
Top-1,e0a319a29d32b8c4decb371b9aba0c96,DOID:305,c01c5c0f2cceb442abcd8e2a80fa68bb8639e8e60d3763...,hypopharynx,UBERON:0001051,XPR017_BICR6.311_96H:A21,carcinoma,1,L1000_LINCS_DCIC_XPR017_BICR6.311_96H_A21_GRB2...,XPR017_BICR6.311_96H_A21_GRB2,...,-10.926251,1.157793e-319,20.611405,9.685154,0.0,6.946711e-319,0,6.946711e-319,1,NaN
Top-2,56ad53c165a443260f29c36575549f04,DOID:3069,267ee507152b7994f1345b2eca7148fc77bc2d8dd60223...,brain,UBERON:0000955,XPR023_U251MG.311_96H:J07,astrocytoma,1,L1000_LINCS_DCIC_XPR023_U251MG.311_96H_J07_SLC...,XPR023_U251MG.311_96H_J07_SLC26A1,...,-14.249436,2.282223e-06,19.923410,5.673974,0.0,6.946711e-319,1,1.967276e-03,1,NaN
Top-3,14166e00cb1900a71eba563f53d5e788,DOID:3459,d159fb03a119165252d162ff0ec6205bc2437f96d202c2...,breast,UBERON:0000310,XPR020_MCF7.311_96H:L23,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR020_MCF7.311_96H_L23_MILR1...,XPR020_MCF7.311_96H_L23_MILR1,...,-12.641875,3.969907e-09,19.656832,7.014957,0.0,6.946711e-319,2,3.255324e-07,1,NaN
Top-4,178e5200556a39b2220430b26542746e,DOID:1324,e0d91544e069810b572973e55bc46688ffaa9c93f7ad12...,lung,UBERON:0002048,XPR013_A549.311_96H:J08,lung cancer,1,L1000_LINCS_DCIC_XPR013_A549.311_96H_J08_CD300...,XPR013_A549.311_96H_J08_CD300A,...,-12.864471,5.263071e-08,19.384283,6.519811,0.0,6.946711e-319,3,9.947204e-06,1,NaN
Top-5,77c819d6800cd226f2d6dcffa814d77f,DOID:3459,72e3896c1cbaab811fe5f48d5ac40af2067f344c27838a...,breast,UBERON:0000310,XPR036_MCF7.311_96H:H18,breast adenocarcinoma,1,L1000_LINCS_DCIC_XPR036_MCF7.311_96H_H18_ASNS.tsv,XPR036_MCF7.311_96H_H18_ASNS,...,-13.333611,5.239208e-07,19.349451,6.015840,0.0,6.946711e-319,4,2.525298e-04,1,NaN


500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/c54dcd80-f537-5d63-86f4-b80634a93f5e
Top-10 Accuracy: False
Top-30 Accuracy: False
Top-50 Accuracy: False
Top-100 Accuracy: False


,md5,doid,sha256,tissue,anatomy,cmap_id,disease,version,filename,local_id,...,uuid,z-down,fdr-up,z-sum,z-up,p-down,p-down-bonferroni,rank,p-up-bonferroni,direction-down
Top-1,8c7e4cb0eba2164a2a182d57908a89b4,DOID:305,75fe2dd95497f987a277c59b0a884169d9b668dc20f659...,hypopharynx,UBERON:0001051,XPR021_BICR6.311_96H:N03,carcinoma,1,L1000_LINCS_DCIC_XPR021_BICR6.311_96H_N03_RRM1...,XPR021_BICR6.311_96H_N03_RRM1,...,cead3954-8142-5402-8178-ea6185c6ddd4,-13.349113,1.300839e-11,21.357065,8.007951,0.0,6.946711e-319,0,1.561007e-10,1
Top-2,a7a1a86a8966438fce889b89deef5269,DOID:305,f1bc40193b2748479a69bf5516aa53152ae95c56f9d9ff...,hypopharynx,UBERON:0001051,XPR021_BICR6.311_96H:I01,carcinoma,1,L1000_LINCS_DCIC_XPR021_BICR6.311_96H_I01_NPSR...,XPR021_BICR6.311_96H_I01_NPSR1,...,1e9bbd37-bbe7-5f3e-824f-baa7b914f652,-13.034854,3.902517e-12,21.280484,8.245629,0.0,6.946711e-319,1,3.122014e-11,1
Top-3,59ea7c564d09efa8a0d18bf3f8460c9e,DOID:3069,8c877409c997d04ef66ac7008ad86db5c87b914cd59225...,brain,UBERON:0000955,XPR030_U251MG.311_96H:N08,astrocytoma,1,L1000_LINCS_DCIC_XPR030_U251MG.311_96H_N08_HSP...,XPR030_U251MG.311_96H_N08_HSPA5,...,75315367-6aae-597c-b595-fd40d26f0f2b,-13.817537,5.567777e-09,20.740995,6.923457,0.0,6.946711e-319,2,6.235910e-07,1
Top-4,e6104bad6a9a3998f9e087ad7d52e6de,DOID:1909,6146e8735775ed7f6697bd9792a6589d379058bed3cd79...,skin of body,UBERON:0002097,XPR021_A375.311_96H:B03,melanoma,1,L1000_LINCS_DCIC_XPR021_A375.311_96H_B03_CCNA2...,XPR021_A375.311_96H_B03_CCNA2,...,b792ab2c-a156-599d-96ed-8d937ca93089,-12.540380,1.561007e-11,20.497587,7.957207,0.0,6.946711e-319,3,2.497611e-10,1
Top-5,53d1b8af05de3c65fa504024880113e1,DOID:1909,2e039f5bd8c329f2423ea231972534c61689eb617feb55...,skin of body,UBERON:0002097,XPR019_A375.311_96H:N06,melanoma,1,L1000_LINCS_DCIC_XPR019_A375.311_96H_N06_SMG1.tsv,XPR019_A375.311_96H_N06_SMG1,...,c1872a59-85eb-5552-b5d7-9ab49260d921,-13.721415,2.074428e-08,20.389178,6.667763,0.0,6.946711e-319,4,3.671738e-06,1


In [4]:
import pandas as pd
benchmark_df = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/benchmark_Norman.csv", index_col=0)

In [5]:
benchmark_df

,SNAI1,CBL_TGFBR2,CBL,CEBPB,CEBPE,CNN1_MAPK1,FEV_CBFA2T3,GLB1L2,LHX1_ELMSAN1,MAP2K6,OSR2,RUNX1T1,S1PR2,SET_KLF1,TGFBR2_IGDCC3
Top-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Top-100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Top-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Top-50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
"LINCS consensus gene (CGS) knockdown signatures"

In [486]:
deg = pd.read_csv(f"{CONDITION}.csv")
try: 
    deg = deg.drop(['Unnamed: 0'], axis=1)
except:
    pass
column = deg.columns[1]
deg_up = deg.sort_values(f"{column}", ascending=False).dropna(axis=0).set_index("gene_name")
deg_down = deg.sort_values(f"{column}", ascending=True).dropna(axis=0).set_index("gene_name")

In [487]:
up_genes = list(deg_up.iloc[:600].index.values)
down_genes = list(deg_down.iloc[:600].index.values)

In [488]:
import requests
import json

METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

input_gene_set = {
    "up_genes": up_genes,
    "down_genes": down_genes
}

all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": all_genes
            }
        },
        "fields": ["id", "meta.symbol"]
    }
}
res = requests.post(METADATA_API + "entities/find", json=payload)
entities = res.json()

for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

for e in entities:
    symbol = e["meta"]["symbol"]
    if symbol in input_gene_set["up_genes"]:
        for_enrichment["up_entities"].append(e["id"])
    elif symbol in input_gene_set["down_genes"]:
        for_enrichment["down_entities"].append(e["id"])
for_enrichment['up_entities'] = for_enrichment['up_entities'][:500]
for_enrichment['down_entities'] = for_enrichment['down_entities'][:500]
len(for_enrichment['up_entities']), len(for_enrichment['down_entities'])

(500, 500)

In [489]:
payload = {
    "meta": {
        "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
        **for_enrichment
    },
    "type": "signature"
}
res = requests.post(METADATA_API + "user_input", json=payload)
persistent_id = res.json()["id"]
print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/01cab592-416b-57e8-84e7-7f901ce248b0


In [1]:
import requests
import json

API_URL = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/listdata"

res = requests.post(API_URL)
databases = res.json()

for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

print(json.dumps(databases, indent=2))

{
  "repositories": [
    {
      "datatype": "rank_matrix",
      "uuid": "gtex_age_sigs"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "LINCS chemical perturbagen signatures"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "human_GEO"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "LINCS consensus gene (CGS) knockdown signatures"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "l1000_mean_xpr"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "l1000_lig"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "LINCS gene overexpression signatures"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "l1000_mean_cp"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "l1000_cp"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "l1000_aby"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "l1000_xpr"
    },
    {
      "datatype": "rank_matrix",
      "uuid": "l1000_oe"
    },
    {
    

In [491]:
"LINCS consensus gene (CGS) knockdown signatures"

'LINCS consensus gene (CGS) knockdown signatures'

In [492]:
NUM_CASE = 120
query = {
    **for_enrichment,
    "limit": NUM_CASE,
    "database": "l1000_xpr"
}

res = requests.post(DATA_API + "enrich/ranktwosided", json=query)
results = res.json()

# Optional, multiply z-down and direction-down with -1
for i in results["results"]:
    i["z-down"] = -i["z-down"]
    i["direction-down"] = -i["direction-down"]

# mimickers = [x for x in results['results'] if x['type']=='mimickers']

In [493]:
# sigids = {i["uuid"]: i for i in mimickers}
results = results['results'][:NUM_CASE]
sigids = {i['uuid']: i for i in results}

payload = {
    "filter": {
        "where": {
            "id": {
                "inq": list(sigids.keys())
            }
        }
    }
}

res = requests.post(METADATA_API + "signatures/find", json=payload)
signatures = res.json()

## Merge the scores and the metadata
for sig in signatures:
    uid = sig["id"]
    scores = sigids[uid]
    # scores.pop("uuid")
    sig["scores"] = scores

print(json.dumps(signatures[0:2], indent=2))

[
  {
    "$validator": "/dcic/signature-commons-schema/v5/core/signature.json",
    "id": "02b7adc7-ba33-5f12-b9cc-27c7536dd6c3",
    "library": "96c7b8c5-1eca-5764-88e4-e4ccaee6603f",
    "meta": {
      "md5": "d6b59d8b8fd6b8c2748cd22866be1eb0",
      "doid": "DOID:3459",
      "sha256": "6c551efbcfb6dc086fb607b6cbf6beefbb29bfa0116d6a641c73f729be307fa4",
      "tissue": "breast",
      "anatomy": "UBERON:0000310",
      "cmap_id": "XPR024_MCF7.311_96H:F22",
      "disease": "breast adenocarcinoma",
      "version": 1,
      "filename": "L1000_LINCS_DCIC_XPR024_MCF7.311_96H_F22_SOCS4.tsv",
      "local_id": "XPR024_MCF7.311_96H_F22_SOCS4",
      "cell_line": "MCF7",
      "pert_name": "SOCS4",
      "pert_time": "96 h",
      "pert_type": "CRISPR Knockout",
      "$validator": "https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/l1000_signatures.json",
      "data_level": 5,
      "creation_time": "2021-05-24",
      "persistent_id": "https://lincs-dcic.s3.amazon

In [494]:
total = {}

for i in signatures:
    dict1 = i['meta']
    dict2 = i['scores']

    dict1.update(dict2)
    
    # break

In [495]:
df_mimickers = pd.DataFrame([x['meta'] for x in signatures])
file_name = f'Norman_K562_{CONDITION}'
# df_mimickers.index = [f'{file_name}_{x}' for x in range(1, 61)]

In [496]:

df_mimickers = df_mimickers.sort_values(by=['z-sum'], ascending=False)
df_mimickers.index = [f'Top-{x}' for x in range(1, len(df_mimickers)+1)]
df_mimickers.dropna(subset=['pert_name'], inplace=True)
df_mimickers
# df_mimickers.dropna(subset=['pert_name'], ).resbet_index(drop=True, )
# df_mimickers.dropna(subset=['pert_name'], inplace=True)
# df_mimickers['z-sum']
mimickers_save_path = f"{path}/{file_name}.csv"
df_mimickers.to_csv(mimickers_save_path)

In [498]:
df_mimickers_top10 = df_mimickers.iloc[:10]
df_mimickers_top30 = df_mimickers.iloc[:30]
df_mimickers_top50 = df_mimickers.iloc[:50]
df_mimickers_top100 = df_mimickers.iloc[:100]

In [499]:
benchmark = {}

for i, rank in zip([df_mimickers_top10, df_mimickers_top30, df_mimickers_top50, df_mimickers_top100], [10, 30, 50, 100]):
    pert_list = i.pert_name.unique()
    if CONDITION not in benchmark.keys():
        benchmark[CONDITION] = {}
    if CONDITION in pert_list:
        print(f'TOP-{rank} Accuracy: True')
        benchmark[CONDITION][f'Top-{rank}'] = 1
    else: 
        print(f'Top-{rank} Accuracy: False')
        benchmark[CONDITION][f'Top-{rank}'] = 0


Top-10 Accuracy: False
Top-30 Accuracy: False
Top-50 Accuracy: False
Top-100 Accuracy: False


In [500]:
new_benchmark_df = pd.DataFrame(benchmark)
benchmark_df = pd.read_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/benchmark_Norman.csv", index_col=0)
benchmark_df = pd.concat([benchmark_df, new_benchmark_df], axis=1)
benchmark_df.to_csv("/home/hb/python/lof/benchmark/sigCOM_lincs/benchmark_Norman.csv", index=True)